In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install lxml

from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
url = 'https://services.nvd.nist.gov/rest/json/cves/2.0?cpeName=cpe:2.3:o:microsoft:windows_11_21h2:10.0.22000&cvssV3Severity=CRITICAL&resultsPerPage=50&startIndex=0&sourceIdentifier=secure@microsoft.com'
response = requests.get(url)
if response.status_code == 200:
    # Parsear la respuesta JSON
    data = response.json()
    
    # Extraer la lista de vulnerabilidades
    vulnerabilities = data.get('vulnerabilities', [])
    cvssMetrics31 = data.get('cvssMetricV31', [])
    
    # Crear una lista para almacenar los datos
    data_list = []
    
    # Iterar sobre cada vulnerabilidad y extraer la información
    for vuln in vulnerabilities:
        cve_data = vuln.get('cve', {})
        cve_id = cve_data.get('id', '')        
        published = cve_data.get('published', '')
        last_modified = cve_data.get('lastModified', '')
        
        # Extremos la Descripción
        descriptions = cve_data.get('descriptions', [])
        description = descriptions[0].get('value', '') if descriptions else ''
            
        # Extramos datos de Métricas CVSS
        metrics = cve_data.get('metrics', {})
                
        # Extraemos datos de CVSS v3.1
        cvss_v31 = metrics.get('cvssMetricV31', [])
        if cvss_v31:
            cvss_v31_data = cvss_v31[0].get('cvssData', {})
            cvss_v31_score = cvss_v31_data.get('baseScore', '')
            cvss_v31_vector = cvss_v31_data.get('vectorString', '')
            attackComplexity = cvss_v31_data.get('attackComplexity', '')
            privilegesRequired = cvss_v31_data.get('privilegesRequired', '')
            userInteraction = cvss_v31_data.get('userInteraction', '')
            scope = cvss_v31_data.get('scope', '')
            confidentiality = cvss_v31_data.get('confidentialityImpact', '')
            integrity = cvss_v31_data.get('integrityImpact', '')
            availability = cvss_v31_data.get('availabilityImpact', '')            
                        
        else:
            cvss_v31_score = ''
            cvss_v31_vector = ''
        
               # Añadir los datos extraídos a una lista
        data_list.append({
            'CVE_ID': cve_id,
            'Published': published,
            'LastModified': last_modified,
            'Description': description,
            'CVSSv3.1_Score': float(cvss_v31_score),
            'CVSSv3.1_Vector': cvss_v31_vector,
            'attackComplexity': attackComplexity,
            'privilegesRequired': privilegesRequired,
            'userInteraction': userInteraction,
            'scope': scope,
            'confidentialityImpact': confidentiality,
            'integrityImpact': integrity,
            'availabilityImpact': availability            
        })
    
    # Creamos un DataFrame con pandas
    df = pd.DataFrame(data_list)
    
    # Seleccionamos las primeras 10 filas y 13 columnas
    src = df.iloc[:10, :13]

    print(df.isnull().sum()) #Verificar si hay valores nulos
    
    #Comprobamos el tipo de datos de la columna CVE_ID
    tipo_dato = df['CVE_ID'].dtype
    print(f"El tipo de dato de la columna es: {tipo_dato}")
    tipos_elementos = df['CVE_ID'].apply(type).value_counts()
    print("Tipos de datos en la columna 'CVE_ID':")
    print(tipos_elementos)
    
    # Si queremos mostrar los datos extraídos de NIST descomentar la siguiente línea
    print(src.head())
else:
    print(f"Error: {response.status_code}")
    print(response.text)




CVE_ID                   0
Published                0
LastModified             0
Description              0
CVSSv3.1_Score           0
CVSSv3.1_Vector          0
attackComplexity         0
privilegesRequired       0
userInteraction          0
scope                    0
confidentialityImpact    0
integrityImpact          0
availabilityImpact       0
dtype: int64
El tipo de dato de la columna es: object
Tipos de datos en la columna 'CVE_ID':
CVE_ID
<class 'str'>    21
Name: count, dtype: int64
           CVE_ID                Published             LastModified  \
0  CVE-2023-21708  2023-03-14T17:15:11.933  2024-11-21T07:43:28.913   
1  CVE-2023-23392  2023-03-14T17:15:12.793  2024-11-21T07:46:05.667   
2  CVE-2023-23415  2023-03-14T17:15:14.857  2024-11-21T07:46:08.937   
3  CVE-2023-21554  2023-04-11T21:15:15.597  2024-11-21T07:43:04.183   
4  CVE-2023-28250  2023-04-11T21:15:24.987  2024-11-21T07:54:41.770   

                                         Description  CVSSv3.1_Score  \
0  R

In [ ]:
#******* Llenar datos en BDD vulnerabilidades y tabla NIst_CVEs
import psycopg2
import json
from datetime import datetime

# Paso 2: Insertar datos estructurados en la tabla
# Conectar a la base de datos    
conn = psycopg2.connect(
    database="vulnerabilidades",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()
       
# Insertar registros en la tabla
insert_query = """
INSERT INTO NIST_CVEs
(
    cve_id,
    Published,
    LastModified,
    Description,
    cvss_v31_score,
    attackVector,
    attackComplexity,
    privilegesRequired,
    userInteraction,
    scope,
    confidentialityImpact,
    integrityImpact,
    availabilityImpact
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
# Preparar los datos como una lista de tuplas
data_to_insert = [tuple(row) for index, row in src.iterrows()]

# Ejecutar insert de múltiples filas a la vez usando executemany
cursor.executemany(insert_query, data_to_insert)

# Confirmar la transacción
conn.commit()
# Cerrar el cursor y la conexión
cursor.close()
conn.close()

print("¡Datos insertados correctamente!")



¡Datos insertados correctamente!


In [14]:
conn = psycopg2.connect(
    database="vulnerabilidades",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()
cursor.execute("SELECT * FROM nist_cves;")

# Obtener todos los resultados de la consulta
rows = cursor.fetchall()

# Mostrar los resultados como un DataFrame de pandas (opcional)
columns = [desc[0] for desc in cursor.description]  # Extraer los nombres de las columnas
df = pd.DataFrame(rows, columns=columns)

# Cerrar el cursor y la conexión
cursor.close()
conn.close()

# Mostrar el DataFrame con los resultados
df

,id,cve_id,published,lastmodified,description,cvss_v31_score,attackvector,attackcomplexity,privilegesrequired,userinteraction,scope,confidentialityimpact,integrityimpact,availabilityimpact
0,1,CVE-2023-21708,2023-03-14T17:15:11.933,2024-11-21T07:43:28.913,Remote Procedure Call Runtime Remote Code Exec...,9.8,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H,LOW,NONE,NONE,UNCHANGED,HIGH,HIGH,HIGH
1,2,CVE-2023-23392,2023-03-14T17:15:12.793,2024-11-21T07:46:05.667,HTTP Protocol Stack Remote Code Execution Vuln...,9.8,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H,LOW,NONE,NONE,UNCHANGED,HIGH,HIGH,HIGH
2,3,CVE-2023-23415,2023-03-14T17:15:14.857,2024-11-21T07:46:08.937,Internet Control Message Protocol (ICMP) Remot...,9.8,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H,LOW,NONE,NONE,UNCHANGED,HIGH,HIGH,HIGH
3,4,CVE-2023-21554,2023-04-11T21:15:15.597,2024-11-21T07:43:04.183,Microsoft Message Queuing (MSMQ) Remote Code E...,9.8,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H,LOW,NONE,NONE,UNCHANGED,HIGH,HIGH,HIGH
4,5,CVE-2023-28250,2023-04-11T21:15:24.987,2024-11-21T07:54:41.770,Windows Pragmatic General Multicast (PGM) Remo...,9.8,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H,LOW,NONE,NONE,UNCHANGED,HIGH,HIGH,HIGH
5,6,CVE-2023-24943,2023-05-09T18:15:12.720,2024-11-21T07:48:49.033,Windows Pragmatic General Multicast (PGM) Remo...,9.8,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H,LOW,NONE,NONE,UNCHANGED,HIGH,HIGH,HIGH
6,7,CVE-2023-32056,2023-07-11T18:15:13.843,2024-11-21T08:02:37.493,Windows Server Update Service (WSUS) Elevation...,7.8,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H,LOW,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH
7,8,CVE-2023-32057,2023-07-11T18:15:13.903,2024-11-21T08:02:37.617,Microsoft Message Queuing (MSMQ) Remote Code E...,9.8,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H,LOW,NONE,NONE,UNCHANGED,HIGH,HIGH,HIGH
8,9,CVE-2023-33154,2023-07-11T18:15:14.737,2024-11-21T08:04:59.827,Windows Partition Management Driver Elevation ...,7.8,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H,LOW,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH
9,10,CVE-2023-35365,2023-07-11T18:15:19.850,2024-11-21T08:08:18.890,Windows Routing and Remote Access Service (RRA...,9.8,CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H,LOW,NONE,NONE,UNCHANGED,HIGH,HIGH,HIGH
